### Step 1 - Start Spark Session, include additional configurations and common functions, and read data from the spark-warehouse database

In [ ]:
%run "../includes/configurations"

In [ ]:
%run "../includes/common_functions"

In [ ]:
appName = "Circuits Ingestion"
spark = create_spark_context(appName, warehouse_location)

In [ ]:
circuits_df = spark.read.parquet(f"{warehouse_location}/f1_raw.db/circuits")

### Step 2 - Rename and drop columns, and add new columns

In [ ]:
circuits_column_mapping = {
    "circuitId": "circuit_id",
    "circuitRef": "circuit_ref",
    "lat": "latitude",
    "lng": "longitude",
    "alt": "altitude"
}

circuits_df = rename_columns(circuits_df, circuits_column_mapping)

In [ ]:
columns_to_drop = ['url']

circuits_df = drop_columns(circuits_df, columns_to_drop)

In [ ]:
circuits_df = add_ingestion_date(circuits_df)

### Step 3 - Write data to a Datalake as a Parquet

In [ ]:
circuits_df.write.mode("overwrite").parquet(f"{processed_folder_path}/circuits")